In [91]:
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline 

In [92]:
# to see all the columns when using head() or describe()
pd.set_option('display.max_columns',60)

In [93]:
df = pd.read_csv("kenya_data/diaries_transactions_all.csv", dtype={'account_startclose_balance': str})

## For now I keep only "business and agriculture loans", "joint liability loans", "personal loans", "borrowing from an informal group", "borrowing from family and friends", "shylock loans".

In [95]:
dfb = df[((df["trx_family_code"]=="FRMLN")&((df["trx_type_code"]==2760)|(df["trx_type_code"]==3247)|(df["trx_type_code"]==2762)))|((df["trx_family_code"]=="INFGRP")&(df["trx_type_code"]==3395))|((df["trx_family_code"]=="INFP2P")&((df["trx_type_code"]==2752)|(df["trx_type_code"]==3246)))]

# 1. Get rid of loans of with unexplained balance adjustments

In [96]:
dfb["trx_prx_purpose"].unique()

array(['7. Closing Balance--End of last DQ',
       '6. Closing Balance--End of last DQ',
       '1. Starting balance (today)', '5. Interest accruing',
       'DECREASE--Unexplained balance adjustment',
       'INCREASE--Unexplained balance adjustment', '4. Any known fees',
       '3. Payments', '6. Loan written off', '6. Debt written off',
       '2. New borrowinng (individual portion)', '2. New borrowing',
       '7. Refund from lender'], dtype=object)

We insert a new column that is 0 if there is no unexplained balance adjustment for this account and a 1 if there is one. The column in 1 or 0 for all entries that correspond to that account.

In [97]:
dfb.insert(dfb.shape[1], "acc_unexplained_trx", np.zeros(dfb.shape[0]))

This program goes through all unique accounts, checks whether it has an unexplained balance adjustment and if yes sets the variable u to 1. Then it goes through another for-loop and sets the entries of that column 0 for all accounts that have an unexplained balance adjustment.

In [98]:
for acc in dfb["account_ids"].unique():
    u = 0
    for index,row in dfb[dfb["account_ids"]==acc].iterrows():
        if row["trx_prx_purpose"]=="DECREASE--Unexplained balance adjustment" or row["trx_prx_purpose"]=="INCREASE--Unexplained balance adjustment":
            u = 1
    for index,row in dfb[dfb["account_ids"]==acc].iterrows():
        dfb.at[index,"acc_unexplained_trx"] = u

### Let us check if it worked.

In [99]:
s = set()
for acc in dfb[dfb["acc_unexplained_trx"]==0]["account_ids"].unique():
     for i in dfb[dfb["account_ids"]==acc]["trx_prx_purpose"].unique():
            s.add(i)
print(s)

{'3. Payments', '6. Loan written off', '2. New borrowinng (individual portion)', '7. Closing Balance--End of last DQ', '1. Starting balance (today)', '4. Any known fees', '7. Refund from lender', '5. Interest accruing', '6. Debt written off', '6. Closing Balance--End of last DQ', '2. New borrowing'}


### "refund from lender" and "Debt written off" might be hard to deal with, but there are only 3 of those

In [103]:
dfb[dfb["trx_prx_purpose"]=="6. Debt written off"]

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,trx_yr_mo,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr,acc_unexplained_trx
3372,KVIHC22,NaN,01oct2012,30sep2013,364,12,04=Diaries Interview,25sep2013,9,2013,2013_09,27aug2012,59134726285800000,NaN,59134362693600000,NaN,Liability,NaN,Informal,NaN,22oct2012,25sep2013,338,11,60138624703300000,59134362693600000,01sep2013,9,2013,2013_09,19.0,335,11,314,10,findev,"Borrowing, lending, savings or insurance media...",INFP2P,Informal P2P,3246,Moneylender or Shylock Borrowing,6. Debt written off,NaN,0.0,Decrease,1.0,Cash,NaN,NaN,NaN,NaN,1700.0,20.0,0,NaN,NaN,NaN,11.55172,0.0


# Let's now only use the loans that have no unexplained balance adjustments.

In [104]:
dfb = dfb[dfb["acc_unexplained_trx"]==0]

# 2. Truncation into chunks with no new borrowing

We first insert a column for the new account numbers. These will be like the old account numbers but with a .1 or .2 for each chunk after a new borrowing.

In [105]:
# this is sometimes necessary if we mess smt up
# del dfb["new_account_ids"]

In [106]:
dfb.insert(df.columns.get_loc("account_ids")+1,"new_account_ids", ["-"]*(dfb.shape[0]))

### Now let's adjust this such that new borrowings are always the first thing on one day, and then run the same program as before.

In [110]:
accs = []
for acc in dfb["account_ids"].unique():
    accs.append(acc)
    i=0
    # we will use "indices" to save the reordered indices of the dataset for each unique loan
    indices = []
    # we will use "index_bor" to save the index of the new borrowing that happens on a given day,
    #whenever there were also other transactions (since we need that index to be first)
    index_bor = 0
    #only keep the rows of that account
    lna = dfb[dfb["account_ids"]==acc]
    # sort the dataset according to time
    lna = lna.sort_values("trx_stdtime_days_acc")
    # for each day in the dataset of the account, we check whether there was 
    # a new borrowing or starting balance and if yes reorder
    for d in lna["trx_stdtime_days_acc"].unique():
        # if only one entry that day we just add the index to indices
        if lna[lna["trx_stdtime_days_acc"]==d].shape[0]==1:
            indices.append(lna[lna["trx_stdtime_days_acc"]==d].index[0])
        # if there are more than 1 entry that day
        elif lna[lna["trx_stdtime_days_acc"]==d].shape[0]>1:
            # if there are more than one new borrowings or starting balances, we need to check manually
            if lna[(lna["trx_stdtime_days_acc"]==d)&((lna["trx_prx_purpose"]=="2. New borrowing")|(lna["trx_prx_purpose"]=="2. New borrowinng (individual portion)")|(lna["trx_prx_purpose"]=="1. Starting balance (today)"))].shape[0]>1:
                print("Several new borrowings on day " + str(d) +", check manually account " + acc + " in category "+ str(lna["trx_type_code"].unique()))
                for i in lna[lna["trx_stdtime_days_acc"]==d].index:
                            indices.append(i)
            #if there is exactly one new borrowing or starting balance that day (here we will need to check for each loan type what they exactly call new borrowings)
            elif lna[(lna["trx_stdtime_days_acc"]==d)&((lna["trx_prx_purpose"]=="2. New borrowing")|(lna["trx_prx_purpose"]=="2. New borrowinng (individual portion)")|(lna["trx_prx_purpose"]=="1. Starting balance (today)"))].shape[0]==1:
                #for each type of new borrowing if there is one that day, put that index first into indices
                for kind in ["2. New borrowing","2. New borrowinng (individual portion)","1. Starting balance (today)"]:
                    if kind in lna[lna["trx_stdtime_days_acc"]==d]["trx_prx_purpose"].unique():
                        index_bor = lna[(lna["trx_stdtime_days_acc"]==d) & (lna["trx_prx_purpose"]==kind)].index[0]
                        indices.append(index_bor)
                        #then add all other indices of that day
                        for i in lna[lna["trx_stdtime_days_acc"]==d].index.drop(index_bor):
                            indices.append(i)
            # if there are several entries for that day but none of them new borrowings just add all indices of that day            
            else:
                for i in lna[lna["trx_stdtime_days_acc"]==d].index:
                    indices.append(i)
    # here we reindex the lna dataset
    lna = lna.loc[indices,]
    #v will be the index of the chunk we are currently at
    v=0
    # saving the old account number
    acc = lna["account_ids"].unique()[0]
    # a is the new account number for each chunk
    a = lna["account_ids"].unique()[0]+".0"
    # we go through all rows of lna and change the entries of the column "new_account_ids" in the original dataset.
    for index, row in lna.iterrows():
        # if there is a new borrowing in the row, increase v by 1 and set a to acc.v
        if row["trx_prx_purpose"] == "1. Starting balance (today)" or row["trx_prx_purpose"] == "2. New borrowinng (individual portion)" or row["trx_prx_purpose"] == "2. New borrowing":
            v +=1
            a = acc+"."+str(v)
            dfb.at[index,"new_account_ids"]=a
        # otherwise set the new account number in that row to a
        else:
             dfb.at[index,"new_account_ids"]=a

Several new borrowings on day 266, check manually account 57137225366300000 in category [2752]
Several new borrowings on day 244, check manually account 61135946960100000 in category [3395]
Several new borrowings on day 0, check manually account 61134787763500000 in category [2752]


Several borrowings on the same day.
57137225366300000
61135946960100000
61134787763500000

In [118]:
df[df["account_ids"]=="57137225366300000"].sort_values("trx_stdtime_days_acc")[["trx_type_desc","trx_stdtime_days_acc","trx_prx_purpose","trx_value_kes","trx_bsheet_direction"]]

,trx_type_desc,trx_stdtime_days_acc,trx_prx_purpose,trx_value_kes,trx_bsheet_direction
476372,Friends and family: Borrowing,0,2. New borrowing,1000.0,Increase
475250,Friends and family: Borrowing,4,2. New borrowing,1000.0,Increase
473295,Friends and family: Borrowing,6,3. Payments,2000.0,Decrease
476508,Friends and family: Borrowing,14,2. New borrowing,1300.0,Increase
475933,Friends and family: Borrowing,38,3. Payments,1300.0,Decrease
475125,Friends and family: Borrowing,175,2. New borrowing,200.0,Increase
475997,Friends and family: Borrowing,182,2. New borrowing,500.0,Increase
474926,Friends and family: Borrowing,188,3. Payments,700.0,Decrease
475005,Friends and family: Borrowing,203,2. New borrowing,600.0,Increase
474162,Friends and family: Borrowing,220,3. Payments,600.0,Decrease


In [114]:
df[df["account_ids"]=="61134787763500000"][["trx_stdtime_days_acc","trx_prx_purpose","trx_value_kes","trx_bsheet_direction"]]

,trx_stdtime_days_acc,trx_prx_purpose,trx_value_kes,trx_bsheet_direction
17354,0,1. Starting balance (today),0.0,Increase
22833,0,1. Starting balance (today),0.0,Increase


The account 61135946960100000 seems to have several new borrowings on day 244.

In [115]:
df[df["account_ids"]=="61135946960100000"][["trx_stdtime_days_acc","trx_prx_purpose","trx_value_kes","trx_bsheet_direction"]]

,trx_stdtime_days_acc,trx_prx_purpose,trx_value_kes,trx_bsheet_direction
5052,285,7. Closing Balance--End of last DQ,3800.0,NaN
43429,141,2. New borrowing,2000.0,Increase
118905,0,2. New borrowing,500.0,Increase
195059,5,3. Payments,500.0,Decrease
212855,244,2. New borrowing,1300.0,Increase
295268,244,2. New borrowing,500.0,Increase


### Should we just get rid of these accounts?

In [111]:
dfb = dfb[(dfb["account_ids"]!="57137225366300000")&(dfb["account_ids"]!="61135946960100000")&(dfb["account_ids"]!="61134787763500000")]

## Let's check if it worked.

Let's see if all accounts got a new name.

In [119]:
dfb[dfb["new_account_ids"]=="-"]

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,new_account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,trx_yr_mo,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr,acc_unexplained_trx


### Let's look at the truncation of a few family and friends borrowings.

In [123]:
for i in range(0,20):
    acc = random.choice(dfb[(dfb["trx_type_code"]==2752)]["account_ids"].unique())
    display(dfb[dfb["account_ids"]==acc].sort_values("trx_stdtime_days_acc")[["trx_type_desc","trx_stdtime_days_acc","trx_value_kes","trx_prx_purpose","account_ids","new_account_ids"]])

,trx_type_desc,trx_stdtime_days_acc,trx_value_kes,trx_prx_purpose,account_ids,new_account_ids
397876,Friends and family: Borrowing,0,3000.0,2. New borrowing,57136617744800000,57136617744800000.1
158943,Friends and family: Borrowing,16,3000.0,3. Payments,57136617744800000,57136617744800000.1
3626,Friends and family: Borrowing,176,0.0,7. Closing Balance--End of last DQ,57136617744800000,57136617744800000.1


,trx_type_desc,trx_stdtime_days_acc,trx_value_kes,trx_prx_purpose,account_ids,new_account_ids
422313,Friends and family: Borrowing,0,3000.0,2. New borrowing,63135523008000000,63135523008000000.1
130218,Friends and family: Borrowing,47,3000.0,3. Payments,63135523008000000,63135523008000000.1
16888,Friends and family: Borrowing,332,0.0,7. Closing Balance--End of last DQ,63135523008000000,63135523008000000.1


,trx_type_desc,trx_stdtime_days_acc,trx_value_kes,trx_prx_purpose,account_ids,new_account_ids
158562,Friends and family: Borrowing,0,300.0,2. New borrowing,65137930163300000,65137930163300000.1
8560,Friends and family: Borrowing,17,300.0,7. Closing Balance--End of last DQ,65137930163300000,65137930163300000.1


,trx_type_desc,trx_stdtime_days_acc,trx_value_kes,trx_prx_purpose,account_ids,new_account_ids
428866,Friends and family: Borrowing,0,20000.0,2. New borrowing,108137416086400000,108137416086400000.1
10682,Friends and family: Borrowing,76,20000.0,7. Closing Balance--End of last DQ,108137416086400000,108137416086400000.1


,trx_type_desc,trx_stdtime_days_acc,trx_value_kes,trx_prx_purpose,account_ids,new_account_ids
3746,Friends and family: Borrowing,0,4000.0,1. Starting balance (today),60134725801400000,60134725801400000.1
359116,Friends and family: Borrowing,30,4000.0,3. Payments,60134725801400000,60134725801400000.1
18303,Friends and family: Borrowing,358,0.0,7. Closing Balance--End of last DQ,60134725801400000,60134725801400000.1


...

,trx_type_desc,trx_stdtime_days_acc,trx_value_kes,trx_prx_purpose,account_ids,new_account_ids
3175,Friends and family: Borrowing,0,700.0,1. Starting balance (today),60136715715500000,60136715715500000.1
186377,Friends and family: Borrowing,218,700.0,3. Payments,60136715715500000,60136715715500000.1
2439,Friends and family: Borrowing,399,0.0,7. Closing Balance--End of last DQ,60136715715500000,60136715715500000.1


,trx_type_desc,trx_stdtime_days_acc,trx_value_kes,trx_prx_purpose,account_ids,new_account_ids
27593,Friends and family: Borrowing,0,2000.0,2. New borrowing,89137995323700000,89137995323700000.1
134185,Friends and family: Borrowing,27,2000.0,3. Payments,89137995323700000,89137995323700000.1
19001,Friends and family: Borrowing,46,0.0,7. Closing Balance--End of last DQ,89137995323700000,89137995323700000.1


,trx_type_desc,trx_stdtime_days_acc,trx_value_kes,trx_prx_purpose,account_ids,new_account_ids
77754,Friends and family: Borrowing,0,500.0,2. New borrowing,105137637052200000,105137637052200000.1
321017,Friends and family: Borrowing,7,500.0,3. Payments,105137637052200000,105137637052200000.1
6193,Friends and family: Borrowing,48,0.0,7. Closing Balance--End of last DQ,105137637052200000,105137637052200000.1


,trx_type_desc,trx_stdtime_days_acc,trx_value_kes,trx_prx_purpose,account_ids,new_account_ids
89166,Friends and family: Borrowing,0,800.0,3. Payments,62134702499000000,62134702499000000.0
483114,Friends and family: Borrowing,77,2030.0,2. New borrowing,62134702499000000,62134702499000000.1
152262,Friends and family: Borrowing,147,1200.0,3. Payments,62134702499000000,62134702499000000.1
337685,Friends and family: Borrowing,254,600.0,2. New borrowing,62134702499000000,62134702499000000.2
307257,Friends and family: Borrowing,301,600.0,3. Payments,62134702499000000,62134702499000000.2
352156,Friends and family: Borrowing,396,30.0,3. Payments,62134702499000000,62134702499000000.2
2602,Friends and family: Borrowing,406,0.0,7. Closing Balance--End of last DQ,62134702499000000,62134702499000000.2


,trx_type_desc,trx_stdtime_days_acc,trx_value_kes,trx_prx_purpose,account_ids,new_account_ids
439986,Friends and family: Borrowing,0,1027.0,2. New borrowing,89137578416400000,89137578416400000.1
440025,Friends and family: Borrowing,11,1027.0,3. Payments,89137578416400000,89137578416400000.1
11353,Friends and family: Borrowing,97,0.0,7. Closing Balance--End of last DQ,89137578416400000,89137578416400000.1


In [124]:
dfb[dfb["account_ids"]=="112137283351600000"]

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,new_account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,trx_yr_mo,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr,acc_unexplained_trx
252,KMOMK03,NaN,07sep2012,25oct2013,413,14,04=Diaries Interview,10oct2013,10,2013,2013_10,01sep2012,112137283351600000,112137283351600000.1,NaN,61134406652800000,NaN,Liability,Close,Formal,NaN,20jun2013,10oct2013,112,3,112138138546700000,61134406652800000,10oct2013,10,2013,2013_10,19.0,398,13,112,3,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,3247,Joint liability loan,6. Closing Balance--End of last DQ,NaN,0.0,NaN,19.0,CLOSING BALANCE,NaN,NaN,NaN,NaN,51200.0,602.35300,0,NaN,NaN,NaN,13.724140,0.0
20371,KMOMK03,NaN,07sep2012,25oct2013,413,14,04=Diaries Interview,03jul2013,7,2013,2013_07,01sep2012,112137283351600000,112137283351600000.1,1.0,61134406652800000,NaN,Liability,NaN,Formal,NaN,20jun2013,10oct2013,112,3,112137283415200000,61134406652800000,20jun2013,6,2013,2013_06,13.0,286,9,0,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,3247,Joint liability loan,4. Any known fees,NaN,1.0,Increase,16.0,Automatic,NaN,NaN,NaN,NaN,1200.0,14.11765,0,NaN,NaN,NaN,9.862069,0.0
236977,KMOMK03,NaN,07sep2012,25oct2013,413,14,04=Diaries Interview,19jul2013,7,2013,2013_07,01sep2012,112137283351600000,112137283351600000.1,NaN,61134406652800000,NaN,Liability,NaN,Formal,NaN,20jun2013,10oct2013,112,3,112137431148400000,61134406652800000,15jul2013,7,2013,2013_07,14.0,311,10,25,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,3247,Joint liability loan,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,1.0,0.00000,07=Group meeting,Outflow,6000.0,70.58823,0,NaN,NaN,NaN,10.724140,0.0
320008,KMOMK03,NaN,07sep2012,25oct2013,413,14,04=Diaries Interview,29aug2013,8,2013,2013_08,01sep2012,112137283351600000,112137283351600000.1,NaN,61134406652800000,NaN,Liability,NaN,Formal,NaN,20jun2013,10oct2013,112,3,112137831140900000,61134406652800000,26aug2013,8,2013,2013_08,17.0,353,12,67,2,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,3247,Joint liability loan,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,1.0,0.00000,"01=Bank, MFI, SACCO, insurance branch",Outflow,4000.0,47.05882,0,NaN,NaN,NaN,12.172410,0.0
443714,KMOMK03,NaN,07sep2012,25oct2013,413,14,04=Diaries Interview,03jul2013,7,2013,2013_07,01sep2012,112137283351600000,112137283351600000.1,NaN,61134406652800000,NaN,Liability,NaN,Formal,NaN,20jun2013,10oct2013,112,3,112137283404000000,61134406652800000,20jun2013,6,2013,2013_06,13.0,286,9,0,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,3247,Joint liability loan,2. New borrowinng (individual portion),New borrowing on financial liability,0.0,Increase,3.0,"Transfer into bank, SACCO, or MFI account",0.0,31.31263,"01=Bank, MFI, SACCO, insurance branch",Inflow,60000.0,705.88230,0,NaN,NaN,NaN,9.862069,0.0


In [125]:
dfb.to_csv("diaries_trx_trunc_loans.csv")

### Careful: the column "unique_accnts" is not accurate anymore.